In [1]:
def get_population_electricity_consumption_table(target_year):
    pass

In [24]:
from os.path import expanduser, join
source_folder = expanduser('~/Projects/infrastructure-planning/datasets')
country_name_variation_table_path = join(source_folder, 'world-country-name-variation.csv')
country_region_income_table_path = join(source_folder, 'world-country-region-income.csv')
electricity_consumption_by_year_table_path = join(source_folder, 'world-electricity-consumption-by-year.csv')
population_by_year_by_country_table_path = join(source_folder, 'world-population-by-year-by-country.csv')

In [26]:
country_name_variation_table = read_csv(country_name_variation_table_path)
country_name_variation_table[:2]

,World Bank,United Nations
0,"Bahamas, The",Bahamas
1,Bolivia,Bolivia (Plurinational State of)


In [15]:
from pandas import read_csv
from StringIO import StringIO
country_region_income_table = read_csv(StringIO(open(country_region_income_table_path, 'r').read().decode('utf-8-sig')))
country_region_income_table[:2]

,Country Name,Country Code,Region,IncomeGroup,SpecialNotes,Unnamed: 5
0,Aruba,ABW,Latin America & Caribbean,High income: nonOECD,SNA data for 2000-2011 are updated from offici...,NaN
1,Afghanistan,AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,NaN


In [16]:
population_by_year_by_country_table = read_csv(population_by_year_by_country_table_path)
population_by_year_by_country_table[:2]

,Country or Area,Year(s),Variant,Value
0,Afghanistan,2100,Const. mortality (Medium),40190.280
1,Afghanistan,2100,Constant-fertility scenario,845843.367


In [22]:
electricity_consumption_by_year_table = read_csv(electricity_consumption_by_year_table_path, skiprows=3)
electricity_consumption_by_year_table[232:234][['Country Name', '2000', '2005', '2010']]

,Country Name,2000,2005,2010
232,Uruguay,2030.264582,1999.634353,2803.153377
233,United States,13671.052024,13704.577048,13394.014607


In [2]:
# List output table columns
# Country, Region, Income Group, Year, Population, Electricity Consumption Per Capita, Electricity Consumption

In [28]:
# Get countries from United Nations
united_nations_countries = demographic_by_year_by_country_table['Country or Area'].unique()
united_nations_countries[:2]

array(['Afghanistan', 'Africa'], dtype=object)

In [31]:
# Get countries from World Bank
world_bank_countries = country_region_income_table['Country Name']
world_bank_countries[:2]

0          Aruba
1    Afghanistan
Name: Country Name, dtype: object

In [32]:
# Select country
united_nations_country = united_nations_countries[0]
united_nations_country

'Afghanistan'

In [41]:
# Get actual and estimated populations for selected country
t = population_by_year_by_country_table
country_t = t[t['Country or Area'] == united_nations_country]
country_t[:2]

,Country or Area,Year(s),Variant,Value
0,Afghanistan,2100,Const. mortality (Medium),40190.280
1,Afghanistan,2100,Constant-fertility scenario,845843.367


In [49]:
# Get the earliest year when the United Nations starts estimating populations
earliest_estimated_year = min(country_t[country_t['Variant'] == 'Low variant']['Year(s)'])
earliest_estimated_year

2010

In [52]:
# Get actual populations for selected country
country_t[country_t['Year(s)'] < earliest_estimated_year][['Country or Area', 'Year(s)', 'Value']]

,Country or Area,Year(s),Value
152,Afghanistan,2005,24860.855
153,Afghanistan,2000,20595.360
154,Afghanistan,1995,17586.073
155,Afghanistan,1990,11731.193
156,Afghanistan,1985,11528.977
157,Afghanistan,1980,13180.431
158,Afghanistan,1975,12551.790
159,Afghanistan,1970,11015.621
160,Afghanistan,1965,9765.015
161,Afghanistan,1960,8774.440


In [ ]:
# Estimate population for the given year

In [ ]:
# Get electricity consumption per capita for provided years

In [ ]:
# Estimate electricity consumption per capita for the given year

In [ ]:
# Multiply population by electricity consumption per capita to get electricity consumption